# Neural Network Base Class

In [1]:
using Random
using SparseArrays

import BSON
import NBInclude: @nbinclude
@nbinclude("Alpha.ipynb");

In [2]:
const device = Flux.gpu;

## Parameters

In [3]:
@with_kw struct Hyperparams
    # model
    implicit::Bool
    activation::String
    # batching
    autoencode::Bool
    batch_size::Int
    input_data::String
    user_sampling_scheme::String
    # optimizer
    l2penalty::Float32
    learning_rate::Float32
    optimizer::String
    # training
    # dropout_perc::Float32
    # dropout_rescale::Bool
    patience::Int
    seed::UInt64
    # loss
    item_weight_decay::Float32
    loss_function::String
    residual_alphas::Vector{String}
    residual_beta::Float32
    user_weight_decay::Float32
end

function to_dict(x::Hyperparams)
    Dict(string(key) => getfield(x, key) for key ∈ fieldnames(Hyperparams))
end

function Base.string(x::Hyperparams)
    fields = [x for x in fieldnames(Hyperparams)]
    max_field_size = maximum(length(string(k)) for k in fields)
    ret = "Hyperparameters:\n"
    for f in fields
        ret *= "$(rpad(string(f), max_field_size)) => $(getfield(x, f))\n"
    end
    ret
end;

# Batching

In [4]:
function SparseArrays.sparse(split::RatingsDataset)
    sparse(split.item, split.user, split.rating, num_items(), num_users())
end

function SparseArrays.sparse(split::RatingsDataset, ratings)
    sparse(split.item, split.user, ratings, num_items(), num_users())
end;

In [5]:
function one_hot_inputs(split)
    @assert !G.autoencode

    # construct inputs
    X = reshape(collect(1:num_users()), (1, num_users()))

    # construct outputs
    Y = sparse(get_split(split; implicit = G.implicit))

    # construct residuals
    residuals = read_alpha(G.residual_alphas, split, G.implicit)
    residuals.rating .*= G.residual_beta
    Z = sparse(residuals)

    # construct loss-function weights
    if split == "training"
        weights =
            expdecay(get_counts(split), G.user_weight_decay) .*
            expdecay(get_counts(split; by_item = true), G.item_weight_decay)
    else
        weights = expdecay(get_counts(split), weighting_scheme("inverse"))
    end
    W = sparse(get_split(split), weights)

    X, Y, Z, W
end;

In [6]:
function get_epoch(split)
    if G.input_data == "one_hot"
        return one_hot_inputs(split)
    end
    @assert false
end;

In [7]:
G = Hyperparams(
    # model
    implicit = false,
    activation = "relu",
    # batching
    autoencode = false,
    batch_size = 128,
    input_data = "one_hot",
    user_sampling_scheme = "constant",
    # optimizer
    l2penalty = 0,
    learning_rate = 0.001,
    optimizer = "ADAM",
    # training
    patience = 5,
    seed = 20220524,
    # loss
    item_weight_decay = 0,
    loss_function = "NONE",
    residual_alphas = [],
    residual_beta = 0,
    user_weight_decay = 0,
);

In [8]:
@time X, Y, Z, W = get_epoch("training")

[ Info: 20220525 02:17:12 regression coefficients: Float32[0.0]
Progress: 100%|███████████████████████████| Time: 0:00:00 ( 0.20 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (33.79 ns/it)


 44.430309 seconds (14.05 M allocations: 24.979 GiB, 12.27% gc time, 21.26% compilation time)


([1 2 … 1320148 1320149], sparse(Int32[1, 2, 7, 10, 11, 12, 15, 16, 21, 25  …  14822, 14894, 15608, 15614, 15622, 15939, 16073, 16125, 16248, 16829], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1320149, 1320149, 1320149, 1320149, 1320149, 1320149, 1320149, 1320149, 1320149, 1320149], Float32[8.0, 8.0, 9.0, 10.0, 7.0, 9.0, 8.0, 7.0, 8.0, 8.0  …  6.0, 9.0, 9.0, 7.0, 1.0, 8.0, 8.0, 7.0, 9.0, 10.0], 18952, 1320149), sparse(Int32[1, 2, 7, 10, 11, 12, 15, 16, 21, 25  …  14822, 14894, 15608, 15614, 15622, 15939, 16073, 16125, 16248, 16829], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1320149, 1320149, 1320149, 1320149, 1320149, 1320149, 1320149, 1320149, 1320149, 1320149], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 18952, 1320149), sparse(Int32[1, 2, 7, 10, 11, 12, 15, 16, 21, 25  …  14822, 14894, 15608, 15614, 15622, 15939, 16073, 16125, 16248, 16829], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1320149, 1320149, 1320149, 1320149, 1320149, 1320149, 1320

In [9]:
function get_sampling_order(split)
    weighting_scheme = split == "training" ? G.user_sampling_scheme : "constant"
    if weighting_scheme == "constant"
        return shuffle(1:num_users())
    else
        weights = expdecay(
            get_counts(split; per_rating = false),
            weighting_scheme(G.user_sampling_scheme),
        )
        return sample(1:num_users(), Weights(weights), num_users())
    end
end;

In [10]:
function get_batch(epoch, iter, batch_size, sampling_order)
    range = sampling_order[(iter-1)*batch_size+1:min(iter * batch_size, size(epoch[1])[2])]
    process(x) = collect(x[:, range]) |> device
    [process.(epoch)]
end;

function get_batch(epoch, iter, batch_size)
    sampling_order = 1:size(epoch[1])[2]
    get_batch(epoch, iter, batch_size, sampling_order)
end;

## Model

In [11]:
function generate_model()
    # inputs are the user's ratings for all shows (unseen shows get mapped to zero) + implicit ratings + heterogenous features
    # outputs are the user's ratings for all shows (unseen shows get mapped to zero), implicit ratings
    # we will train ratings using mse on observed shows, and implicit ratings via crossentropy loss
    n_inputs = n_items + n_items + (G.use_derived_features ? 9 : 0)
    layers = [[n_inputs]; G.layers; [n_items]]
    if G.activation == "relu"
        activation = relu
    else
        @assert false
    end
    autoencoder = [Dense(layers[i], layers[i+1], activation) for i = 1:length(layers)-1]
    m = Chain(Dropout(G.dropout_perc), autoencoder...) |> device
    m |> device
end;

## Loss Functions

In [12]:
function evaluate(m, split)
    BLAS.set_num_threads(1)

    df = get_split(split)
    users = df.user
    items = df.item

    # index users
    user_to_output_idxs = [Dict() for t = 1:Threads.nthreads()]
    @tprogress Threads.@threads for j = 1:length(users)
        u = users[j]
        t = Threads.threadid()
        if u ∉ keys(user_to_output_idxs[t])
            user_to_output_idxs[t][u] = []
        end
        push!(user_to_output_idxs[t][u], j)
    end
    user_to_output_idxs = merge(vcat, user_to_output_idxs...)

    # compute predictions
    ratings = zeros(Float32, length(users))
    epoch = get_epoch(split)
    epoch = (epoch[1], epoch[2], collect(1:n_users)')
    batch_size = 16
    @tprogress Threads.@threads for iter = 1:Int(ceil(n_users / batch_size))
        batch = get_batch(epoch, iter, batch_size)[1]
        alpha = m(batch[1]) |> cpu
        if G.train_implicit_model
            alpha .= exp.(alpha)
            alpha .= alpha ./ sum(alpha, dims = 1)
        end

        for j = 1:size(alpha)[2]
            u = batch[3][1, j]
            if u ∉ keys(user_to_output_idxs)
                continue
            end
            for output_idx in user_to_output_idxs[u]
                ratings[output_idx] = alpha[items[output_idx], j]
            end
        end
    end

    BLAS.set_num_threads(Threads.nthreads())

    RatingsDataset(user = users, item = items, rating = ratings)
end;

In [13]:
function rating_loss(ŷ, y, weights)
    # only compute loss on items the user has seen
    mask = y .!= 0
    per_user_mse =
        sum(((ŷ .- y) .* mask) .^ 2, dims = 1) ./
        max.(one(eltype(weights)), sum(mask, dims = 1))
    dot(per_user_mse, weights) / sum(weights)
end

function implicit_loss(ŷ, y, weights)
    agg(x) = dot(x, weights)
    Flux.logitcrossentropy(ŷ, y, agg = agg) / sum(weights)
end

loss(m, x, y, weights) =
    G.train_implicit_model ? implicit_loss(m(x), y, weights) : rating_loss(m(x), y, weights);

In [14]:
function get_loss(m, split)
    BLAS.set_num_threads(1)

    epoch = get_epoch(split)
    batch_size = 16
    losses = zeros(Threads.nthreads())
    @tprogress Threads.@threads for iter = 1:Int(ceil(n_users / batch_size))
        batch = get_batch(epoch, iter, batch_size)
        losses[Threads.threadid()] += loss(m, batch[1]...) * sum(batch[1][3])
    end

    BLAS.set_num_threads(Threads.nthreads())

    sum(losses) / sum(epoch[3])
end;

In [15]:
function get_residualized_loss(m, split)
    rating = evaluate(m, split).rating
    df = get_residuals(split, G.validation_residuals)

    # turn per-user weights into per-item weights
    W = get_derived_feature(
        df,
        (_, count) ->
            weighting_scheme(count, G.validation_weight_scheme) *
            weighting_scheme(count, "inverse"),
    )
    weights = zeros(eltype(rating), length(df.user))
    Threads.@threads for i = 1:length(weights)
        weights[i] = W[df.user[i]]
    end

    if G.train_implicit_model
        @assert false
    else
        Y = df.rating .* sqrt.(weights)
        X = rating .* sqrt.(weights)
        β = X \ Y
        @info "beta: $β"
        return mse(Y, X .* β, weights)
    end
end;

## Training

In [16]:
function checkpoint(m)
    loss_fn = G.use_residualized_validation_loss ? get_residualized_loss : get_loss
    training_loss = loss_fn(m, "training")
    validation_loss = loss_fn(m, "validation")
    @info "training loss $training_loss, validation loss $validation_loss"
    validation_loss
end;

In [17]:
function continue_training(m, stop_criteria, model_path)
    validation_loss = checkpoint(m)
    if validation_loss < stop_criteria.loss
        BSON.@save model_path m
    end
    !stop!(stop_criteria, validation_loss)
end;

In [18]:
function train_epoch!(m, opt; checkpoint_rate = 0.1)
    BLAS.set_num_threads(Threads.nthreads())
    ps = Flux.params(m)
    train_loss(x, y, w) = loss(m, x, y, w)
    epoch = get_epoch("training")
    sampling_order = get_sampling_order("training")

    nbatches = Int(ceil(size(epoch[1])[2] / G.batch_size))
    ProgressMeters.@showprogress for iter = 1:nbatches
        batch = get_batch(epoch, iter, G.batch_size, sampling_order)
        Flux.train!(train_loss, ps, batch, opt)

        if iter % Int(round(nbatches * checkpoint_rate)) == 0
            checkpoint(m)
        end
    end
end;

LoadError: LoadError: UndefVarError: ProgressMeters not defined
in expression starting at In[18]:9

In [19]:
function train_model(hyperparams::Hyperparams)
    # unpack parameters
    global G = hyperparams
    Random.seed!(G.seed)
    m = generate_model()
    if G.optimizer == "ADAM"
        opt = ADAMW(G.learning_rate, (0.9, 0.999), G.l2penalty)
    else
        @assert false
    end
    stop_criteria = early_stopper(patience = G.patience)
    model_path = "../../data/alphas/$name/model.$(hash(G)).bson"

    # Train model
    while continue_training(m, stop_criteria, model_path)
        train_epoch!(m, opt)
    end

    model_path
end;

## Write predictions

In [20]:
function make_prediction(sparse_preds, users, items)
    preds = zeros(length(users))
    @tprogress Threads.@threads for j = 1:length(preds)
        preds[j] = sparse_preds[users[j], items[j]]
    end
    preds
end;

In [21]:
function save_model(model_path, hyperparams::Hyperparams, outdir)
    global G = hyperparams
    BSON.@load model_path m
    training = evaluate(m, "training")
    validation = evaluate(m, "validation")
    test = evaluate(m, "test")
    df = reduce(cat, [training, validation, test])
    sparse_preds = sparse(df.user, df.item, df.rating)

    write_predictions(
        (users, items) -> make_prediction(sparse_preds, users, items),
        residual_alphas = G.validation_residuals,
        outdir = outdir,
        implicit = G.train_implicit_model,
    )
    params = to_dict(G)
    params["model"] = model_path
    write_params(params, outdir = outdir)
end;

In [22]:
function fit(hyperparams::Hyperparams, outdir)
    redirect_logging("../../data/alphas/$outdir")
    @info string(hyperparams)
    model_path = train_model(hyperparams)
    save_model(model_path, hyperparams, outdir)
end;

In [23]:
# hyperparams = Hyperparams(
#     use_derived_features = true,
#     train_implicit_model = true,
#     activation = "relu",
#     autoencode = true,
#     batch_size = 128,
#     dropout_perc = 0.5,
#     dropout_rescale = false,
#     layers = [512, 256, 128, 256, 512],
#     l2penalty = 1e-5,
#     learning_rate = 0.001,
#     optimizer = "ADAM",
#     patience = 10,
#     sampling_weight_scheme = "linear",
#     training_residuals = ["UserItemBiases"],
#     training_weight_scheme = "linear",
#     use_residualized_validation_loss = false,
#     validation_residuals = ["UserItemBiases"],
#     validation_weight_scheme = "constant",
#     seed = 20220501 * hash(name),
# )

# #fit(hyperparams, "GNN.Rating.Test.2")